# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Jayapura,-2.5333,140.7000,78.19,92,68,5.44,ID,2023-10-29 18:01:14
1,1,Bilibino,68.0546,166.4372,-9.42,93,18,4.94,RU,2023-10-29 18:01:09
2,2,Murun-Kuren,49.6342,100.1625,30.72,36,94,2.91,MN,2023-10-29 18:01:14
3,3,Tiksi,71.6872,128.8694,9.14,95,100,5.39,RU,2023-10-29 18:01:15
4,4,Grytviken,-54.2811,-36.5092,38.07,91,74,5.77,GS,2023-10-29 18:01:15


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
city_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color='City'
)
city_plot

c:\Users\bdbag\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:

ideal_city = city_data_df.loc[(city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0) & (city_data_df['Max Temp'] < 90) & (city_data_df['Max Temp'] > 60)] 
ideal_city

# Temperature range increased due to lack of ideal city data


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
99,99,Phalodi,27.1333,72.3667,84.02,27,0,0.83,IN,2023-10-29 18:01:44
151,151,Lompoc,34.6391,-120.4579,74.84,48,0,3.00,US,2023-10-29 18:01:54
292,292,Basni,27.1667,73.6500,79.79,29,0,2.57,IN,2023-10-29 18:02:35
514,514,Paphos,34.7667,32.4167,73.18,53,0,3.44,CY,2023-10-29 18:03:30
552,552,Urangan,-25.3000,152.9000,66.52,64,0,3.78,AU,2023-10-29 18:03:40


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:

hotel_df = pd.DataFrame(ideal_city.drop(columns= ['Date', 'City_ID', 'Max Temp','Cloudiness','Wind Speed']))
hotel_df['Hotel Name'] = ""
hotel_df.reset_index()
hotel_df





,City,Lat,Lng,Humidity,Country,Hotel Name
99,Phalodi,27.1333,72.3667,27,IN,
151,Lompoc,34.6391,-120.4579,48,US,
292,Basni,27.1667,73.6500,29,IN,
514,Paphos,34.7667,32.4167,53,CY,
552,Urangan,-25.3000,152.9000,64,AU,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
hotels = 'accommodation.hotel'

params = {
    "categories":"accommodation.hotel", 
    "apiKey": geoapify_key,
    "limit":20
}
         

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}" 
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?" + geoapify_key

     # Make and API request using the params dictionary
    hotel_query = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    hotel_query = hotel_query.json()

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_query["features"][0]["properties"]["name"]
    except Exception as e:
       print (e)
    
    
    #except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        #hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Phalodi - nearest hotel: LAL NIWAS
Lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
Basni - nearest hotel: Ranvas Nagaur
Paphos - nearest hotel: Mayfair Hotel
Urangan - nearest hotel: Ramada


,City,Lat,Lng,Humidity,Country,Hotel Name
99,Phalodi,27.1333,72.3667,27,IN,LAL NIWAS
151,Lompoc,34.6391,-120.4579,48,US,Embassy Suites by Hilton Lompoc Central Coast
292,Basni,27.1667,73.6500,29,IN,Ranvas Nagaur
514,Paphos,34.7667,32.4167,53,CY,Mayfair Hotel
552,Urangan,-25.3000,152.9000,64,AU,Ramada


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
ideal_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color='City',
    hover_cols = ["City", "Country",'Hotel Name']
)
ideal_plot



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)